In [1]:
import pandas as pd
from LCS import LCS, predicting
# from .mainland_students import Row
# from backend.models import Task

EDU_SYS = {
    "學士" : "學士班",
    "碩士" : "碩士班",
    "碩士專班" : "碩士在職專班",
    "博士" : "博士班"
}

FILTER = ["學制班別", "年級"]

EDU_SYS_DEPTS = {
    
}

STUDENT_HEAD = [
    "college_name",
    "dept_name",
    "education_system",
    "stu_no",
    "stu_name",
    "stu_year",
    "gender",
    "country_name",
    "id_b_remark"
]

class Row:
    
    def __init__(self, configure):
        # print(configure)
        # self.col = col
        self.configure = configure
        self.area = {}
    
    def append(self, student):
        pass



class Table5_1(Row):
    def __init__(self, configure, country_code):
        super().__init__(configure)
        self.country_code = country_code



    def append(self, values:pd.Series):
        country_code = self.country_code[self.country_code["國別"] == values["country_name"]].國別代碼.values[0]
        if not country_code in self.area:
            self.area[country_code] = {}
            self.area[country_code]["low"] = {}
            self.area[country_code]["low"]["boy"] = []
            self.area[country_code]["low"]["girl"] = []

            self.area[country_code]["normal"] = {}
            self.area[country_code]["normal"]["boy"] = []
            self.area[country_code]["normal"]["girl"] = []

            self.area[country_code]["two"] = {}
            self.area[country_code]["two"]["boy"] = []
            self.area[country_code]["two"]["girl"] = []


        if values["id_b_remark"] in ("大學部指考生、研究所招生","甄試生", "學士直升博士", "直攻博士"):
            if values["gender_code"] == 1:
                self.area[country_code]["normal"]["boy"].append(values)
            else:
                self.area[country_code]["normal"]["girl"].append(values)
        elif values["id_b_remark"]  in ("獨招生", "博返碩", "中5生(僑,外生)"):
            if values["gender_code"]== 1:
                self.area[country_code]["low"]["boy"].append(values)
            else:
                self.area[country_code]["low"]["girl"].append(values)
 
        elif values["id_b_remark"] in ("雙聯學制"):
            if values["gender_code"] == 1:
                self.area[country_code]["two"]["boy"].append(values)
            else:
                self.area[country_code]["two"]["girl"].append(values)
               
        else:
            return None

    
    def output(self):
        outputData = []
        for key, item in self.area.items():
            configure = self.configure.copy()
            configure["僑居地國別"] = key
            configure["外國畢業學生人數_依就學辦法來臺_男"] = len(item["low"]["boy"])
            configure["外國畢業學生人數_依就學辦法來臺_女"] = len(item["low"]["girl"])

            configure["外國畢業學生人數_依一般升學管道就學者_男"] = len(item["normal"]["boy"])
            configure["外國畢業學生人數_依一般升學管道就學者_女"] = len(item["normal"]["girl"])

            configure["在臺雙聯學制外國畢業學生人數_男"] = len(item["two"]["boy"])
            configure["在臺雙聯學制外國畢業學生人數_女"] = len(item["two"]["girl"])

            students = []

            for way in item:
                for gender in item[way]:
                    for i in range(len(item[way][gender])):
                        students.append(list(item[way][gender][i].get(STUDENT_HEAD).values))
            # for i in item["boy"]:
            #     students.append(list(i.get(STUDENT_HEAD).values))
                
            # for i in item["girl"]:
            #     students.append(list(i.get(STUDENT_HEAD).values))
            
            outputData.append([list(configure.values), students])
            
        # print(test)
        return outputData

    def outputMainTable(self):
        outputData = []
        for key, item in self.area.items():
            configure = self.configure.copy()
            configure["中文國別(地區)代碼"] = key
            configure["外國畢業學生人數_依就學辦法來臺_男"] = len(item["low"]["boy"])
            configure["外國畢業學生人數_依就學辦法來臺_女"] = len(item["low"]["girl"])

            configure["外國畢業學生人數_依一般升學管道就學者_男"] = len(item["normal"]["boy"])
            configure["外國畢業學生人數_依一般升學管道就學者_女"] = len(item["normal"]["girl"])

            configure["在臺雙聯學制外國畢業學生人數_男"] = len(item["two"]["boy"])
            configure["在臺雙聯學制外國畢業學生人數_女"] = len(item["two"]["girl"])

            outputData.append(configure)
        
        return outputData


# {
#     "學士班" : {
#         "數學系" : {
#             1 : Row()
#         }
#     }
# }

def constructDataStructure(config:pd.DataFrame, country_code):
    data = {}
    for i in range(len(config.values)):
        dept = config.loc[i]["單位"]
        edu_system = config.loc[i]["學制班別"]
#         grade = config.loc[i]["年級"]

        if not edu_system in data:
            data[edu_system] = {}
        
        if not dept in data[edu_system]:
            data[edu_system][dept] = Table5_1(config.loc[i], country_code)
        
#         if not grade in data[edu_system][dept]:
#             data[edu_system][dept][grade] = Table5(config.loc[i])

    # print(data.keys())

    return data



def cleanData(studentData:pd.DataFrame):
    studentData.id_a_remark = studentData.id_a_remark.str.strip(' ')
    studentData.id_b_remark = studentData.id_b_remark.str.strip(' ')
    studentData = studentData[studentData.id_a_remark == '外國學生']
    
    studentData.education_system = studentData.education_system.str.strip(' ')
    studentData.country_name = studentData.country_name.str.strip(' ')
    studentData.dept_name = studentData.dept_name.str.replace(r'\w士班', '')
    studentData.dept_name = studentData.dept_name.str.replace(r'應用數學', '')
    studentData.dept_name = studentData.dept_name.str.replace(r' ', '')
    studentData.stu_name = studentData.stu_name.str.strip(r' ')
    studentData.college_name = studentData.college_name.str.strip(r' ')
    studentData.fillna('', inplace=True)
    return studentData


def init(config):
    global EDU_SYS_DEPTS
    EDU_SYS_DEPTS["學士"] = list(set(config.query("學制班別 == '學士班'").單位))
    EDU_SYS_DEPTS["碩士"] = list(set(config.query("學制班別 == '碩士班'").單位))
    EDU_SYS_DEPTS["博士"] = list(set(config.query("學制班別 == '博士班'").單位))
    EDU_SYS_DEPTS["碩士專班"] = list(set(config.query("學制班別 == '碩士在職專班'").單位))


def main(config, studentData):
#     taskInstance = arguments[0]
#     config = pd.read_excel(taskInstance.output_file.open(), dtype=str)
    country_code = pd.read_excel("./國別代碼對照表 (6).xls")
    config.fillna('', inplace=True)
    dataStructure = constructDataStructure(config, country_code)
#     studentData = pd.read_excel(taskInstance.database_information.open(), dtype=str)
    init(config)
    studentData = cleanData(studentData)
    studentData.reset_index()

    for i in range(len(studentData.values)):
        edu_sys = EDU_SYS[studentData.iloc[i]["education_system"]]
        dept_name = predicting(studentData.iloc[i]["dept_name"], EDU_SYS_DEPTS[studentData.iloc[i]["education_system"]])
#         stu_year = studentData.iloc[i]["stu_year"]
        dataStructure[edu_sys][ dept_name ].append(studentData.iloc[i])

    statisticData = []
    tableData = []
    for i in dataStructure:
        for j in dataStructure[i]:
            statisticData += dataStructure[i][j].output()
            tableData += dataStructure[i][j].outputMainTable()


    newdf = pd.DataFrame(tableData)

    head = list(config.columns)
    head.append('僑居地國別')

    

    return {
        "head" : head,
        "data" : statisticData,
        "sub_head" : STUDENT_HEAD
    }, newdf


In [2]:
config = pd.read_excel("./第10910期_學5-1外國畢業學生資料統計表.xls")

In [3]:
studentData = pd.read_excel("./108畢業生﹍1090924下午for 學4-1 5-1.xlsx", sheet_name=1)

In [4]:
result = main(config, studentData)

In [5]:
result[1].to_excel("output5-1.xlsx")